In [30]:
import trimet_plan_game_helper_scripts as helpers
import geopandas as gpd
import boto3
from shapely.geometry import LineString
import pandas as pd
import seaborn as sns

In [31]:
tm_boundary = gpd.read_file("gis/export/tm_route_buffer_bounds.geojson")
trimet_crs = "EPSG:2913"

In [32]:
gdf_points, itineraries_reduced, itinerary_routes_reduced, tries = helpers.generate_random_points_make_itinerary(tm_boundary, trimet_crs)

In [33]:
itineraries_reduced.head(3)

,itin_idx,totalTime,walkTime,transitTime,waitingTime,walkDistance,leg_id,route_id,mode,fromStopCode,fromName,toStopCode,toName,legGeometry,route_length_deg,route_id_combo
0,0,5351,1776,3322,253,2558.909881,0,WALK,WALK,,Origin,913,SW Canyon & 87th,"LINESTRING (-122.76445 45.50085, -122.76439 45...",0.005868,58 to 44
1,0,5351,1776,3322,253,2558.909881,1,58,BUS,913,SW Canyon & 87th,12793,SW Columbia between 6th & 5th,"LINESTRING (-122.76626 45.49819, -122.76607 45...",0.099323,58 to 44
2,0,5351,1776,3322,253,2558.909881,2,WALK,WALK,12793,SW Columbia between 6th & 5th,11486,SW 6th & Columbia,"LINESTRING (-122.68066 45.51420, -122.68105 45...",0.000464,58 to 44


In [34]:
itinerary_routes_reduced

,itin_idx,totalTime,walkTime,transitTime,waitingTime,walkDistance,leg_id,route_id,mode,fromStopCode,fromName,toStopCode,toName,legGeometry,route_length_deg,route_id_combo
1,0,5351,1776,3322,253,2558.909881,1,58,BUS,913,SW Canyon & 87th,12793,SW Columbia between 6th & 5th,"LINESTRING (-122.76626 45.49819, -122.76607 45...",0.099323,58 to 44
3,0,5351,1776,3322,253,2558.909881,3,44,BUS,11486,SW 6th & Columbia,6298,N Willamette & Woolsey,"LINESTRING (-122.68121 45.51429, -122.68115 45...",0.114430,58 to 44
8,1,5378,789,4295,294,866.941925,3,4,BUS,12791,SW Jefferson & 5th,12725,N Fessenden & Haven,"LINESTRING (-122.67957 45.51474, -122.67965 45...",0.143625,58 to 4


In [35]:
itinerary_routes_reduced['route_id'] = itinerary_routes_reduced['route_id'].astype(int)

In [36]:
gtfs_routes = pd.read_csv("gtfs/routes.txt")
gtfs_routes.head(2)

,route_id,agency_id,route_short_name,route_long_name,route_type,route_url,route_color,route_text_color,route_sort_order
0,1,TRIMET,1,Vermont,3,https://trimet.org/schedules/r001.htm,NaN,NaN,400
1,2,TRIMET,FX2,Division,3,https://trimet.org/schedules/r002.htm,61A60E,FFFFFF,500


In [37]:
itinerary_routes_long_name = itinerary_routes_reduced.merge(gtfs_routes[['route_id','route_long_name']], how='inner', on='route_id')

In [38]:
# Get Unique continents
color_labels = itinerary_routes_long_name['route_id'].unique()
# List of colors in the color palettes
hex_values = sns.color_palette("Set2", len(color_labels)).as_hex()
# Map continents to the colors
color_map = dict(zip(color_labels, hex_values))
color_map['WALK'] = '#bf5f58'

In [39]:
color_map_str = {str(item[0]):item[1] for item in color_map.items()}

In [40]:
color_map.items()

dict_items([(58, '#66c2a5'), (44, '#fc8d62'), (4, '#8da0cb'), ('WALK', '#bf5f58')])

In [41]:
itinerary_routes_long_name['route_color'] = itinerary_routes_long_name['route_id'].apply(lambda x: color_map[x])

In [42]:
itinerary_routes_long_name['dropdown_route'] = itinerary_routes_long_name.apply(lambda x: str(x['route_id'])+" - "+str(x['route_long_name']), axis=1)

In [43]:
itineraries_reduced_with_long_name = itineraries_reduced.merge(gtfs_routes[['route_id','route_long_name']].astype(str), how='left', on='route_id')

In [44]:
itineraries_reduced_with_long_name.head(3)

,itin_idx,totalTime,walkTime,transitTime,waitingTime,walkDistance,leg_id,route_id,mode,fromStopCode,fromName,toStopCode,toName,legGeometry,route_length_deg,route_id_combo,route_long_name
0,0,5351,1776,3322,253,2558.909881,0,WALK,WALK,,Origin,913,SW Canyon & 87th,"LINESTRING (-122.76445 45.50085, -122.76439 45...",0.005868,58 to 44,NaN
1,0,5351,1776,3322,253,2558.909881,1,58,BUS,913,SW Canyon & 87th,12793,SW Columbia between 6th & 5th,"LINESTRING (-122.76626 45.49819, -122.76607 45...",0.099323,58 to 44,Canyon Rd
2,0,5351,1776,3322,253,2558.909881,2,WALK,WALK,12793,SW Columbia between 6th & 5th,11486,SW 6th & Columbia,"LINESTRING (-122.68066 45.51420, -122.68105 45...",0.000464,58 to 44,NaN


In [45]:
itineraries_reduced_with_long_name['route_long_name'] = itineraries_reduced_with_long_name['route_long_name'].fillna(value='WALK')

In [46]:
itineraries_reduced_with_long_name['dropdown_route'] = itineraries_reduced_with_long_name.apply(lambda x: str(x['route_id'])+" - "+str(x['route_long_name']), axis=1)

In [47]:
itineraries_reduced_with_long_name['route_color'] = itineraries_reduced_with_long_name['route_id'].apply(lambda x: color_map_str[x])

In [48]:
gdf_points.to_file("gis/export/origin_destination_points.geojson", driver="GeoJSON")
client = boto3.client("s3")
client.upload_file("gis/export/origin_destination_points.geojson", "meysohn-sandbox", "trimet_trip_planner/origin_destination_points.geojson",ExtraArgs={'ACL':'public-read'})

In [49]:
itineraries_reduced_with_long_name.to_file("gis/export/itineraries_reduced_with_long_name.geojson", driver="GeoJSON")
client = boto3.client("s3")
client.upload_file("gis/export/itineraries_reduced_with_long_name.geojson", "meysohn-sandbox", "trimet_trip_planner/itineraries_reduced_with_long_name.geojson",ExtraArgs={'ACL':'public-read'})

In [50]:
itinerary_routes_long_name.to_file("gis/export/itinerary_routes_long_name.geojson", driver="GeoJSON")
client = boto3.client("s3")
client.upload_file("gis/export/itinerary_routes_long_name.geojson", "meysohn-sandbox", "trimet_trip_planner/itinerary_routes_long_name.geojson",ExtraArgs={'ACL':'public-read'})

In [51]:
origin_destination_centriod = LineString(list(gdf_points['points'].to_numpy())).centroid

In [52]:
origin_destination_centriod_gdf = gpd.GeoDataFrame(pd.DataFrame(['origin_dest_centroid'], columns=['id']),crs="EPSG:4326", geometry=[origin_destination_centriod])
origin_destination_centriod_gdf

,id,geometry
0,origin_dest_centroid,POINT (-122.74106 45.54537)


In [53]:
origin_destination_centriod_gdf.to_file("gis/export/origin_destination_centriod.geojson", driver="GeoJSON")
client = boto3.client("s3")
client.upload_file("gis/export/origin_destination_centriod.geojson", "meysohn-sandbox", "trimet_trip_planner/origin_destination_centriod.geojson",ExtraArgs={'ACL':'public-read'})